<a href="https://colab.research.google.com/github/lpceronm/QKD/blob/master/Quantum_Key_Distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Quantum Key Distribution



Cryptography aims to communicate data between authorized entities in a secure way, in fact the encryption era started a century ago with the *One-Time pad* proposed by Vernam which uses a symmetric one-use key shared between a sender and a receiver and that is able to secure the data as long as the key remains secret.  This is where quantum properties provide a perfectly secret transmission key relying upon the immutable laws of physics and instead of  the computational complexity the way it’s implemented  today.

In a QKD system, first studied by Stephen Weisner (1970s), an eavesdropper could be easily detected thanks to the observation errors its measurements introduce to the quantum channel. Nowadays commercial QKD networks are aimed mainly for governments and corporations with high security requirements, however they are currently limited  to short links specially because of problems in the physical layer.

The basic scheme of a quantum key distribution consists of two* authorized partners* sharing information over a classical channel and establishing the secret key with a quantum connection (usually optical fiber).  The partners are traditionally called **Alice** and **Bob**, in case of a third part trying to intercept the communication it is named **Eve** for eavesdropper.

![](https://i.postimg.cc/rmKK355x/QKD.jpg)

The QKD by deffinition tracks Eve, any attempt to measure modifies the state of the measured system. If the eavesdropper interferes the quantum channel it has to recreate the states in order to send them to Bob since the qubit is destroyed when measured. Protocols define a tolerance threshold for the error rate. Error rate is susceptible to eavesdropping, the effect of Eve is now detectable and communication could be restarted if needed. Alternatively if Eve wants to copy the key duplicating the states for further processing this is proven impossible by the non-clonning theorem (Wootters and Zurek, 1982) which makes the quantum channel robust.

There are three main families of explicit QKD protocols: discrete-variable coding, continuous-variable coding, and more recently distributed phase-reference coding, the most significant difference is the scheme detection. In this project we will focus in the well-known discrete-variable protocol BB84 and some of its variations.   






```
```



In [0]:
import numpy as np
import math as mt
import matplotlib.pyplot as plt
import matplotlib as mpl

In [0]:
# Basis
zero = np.array([1,0])## 0
one = np.array([0,1])## 1
plus = np.array([1,1])/np.sqrt(2)## 0
minus = np.array([1,-1])/np.sqrt(2)## 1
l = (1/np.sqrt(2))*(zero + 1j*one)## 0
r = (1/np.sqrt(2))*(zero - 1j*one)## 1

# Pauli matrices
X = np.array([[0 ,1],[1, 0]])
Y = np.array([[0 ,1j],[-1j, 0]])
Z = np.array([[1 ,0],[0, -1]])

In [0]:
class Alice(object):

  usedBasis = []
  bits = []
  qbits = []
  siftedKey = []

  basis = {
    # Base 0/1
    0: {'0': zero, '1': one },
    # Base +/-
    1: {'0': plus, '1': minus},
    # Base L/R
    2:{ '0':l, '1':r  }     
  }
  
  def genQbits(self, lengthChain):
    self.bits = []
    self.qbits = []
    self.usedBasis = []
    self.bits = [np.random.randint(0, 2) for i in range(lengthChain)]
    for i in self.bits:
      base = np.random.choice(list(self.basis.keys()))
      self.usedBasis.append(base)
      self.qbits.append(self.basis[base][str(i)])  
    return self.qbits
  
  def compareBasis(self, sentBasis):
    self.siftedKey = []
    compared = np.array(self.usedBasis) == np.array(sentBasis)
    for i in range(len(compared)):
      if compared[i] == True: 
        self.siftedKey.append(self.bits[i])
    return compared
  
  def errorCheck(self):
    toCheck = []
    index = []
    numInd = mt.ceil(len(self.siftedKey)* 0.15)
    print(numInd)
    index = np.random.choice(np.array(self.siftedKey).shape[0], 
                             numInd, replace=False) 
    for i in index:
      toCheck.append(self.siftedKey[i])
      self.siftedKey[i] = None  
    self.siftedKey = [x for x in self.siftedKey if x is not None]
    return index,toCheck
  
  def shuffleSiftedKeyKey(self):
    perm = np.random.get_state()
    np.random.shuffle(self.siftedKey)
    return perm

In [0]:
class Bob(object):
  usedBasis = []
  bits = []
  siftedKey = []
  errorRate = 0.0
  
  def __init__(self):
    self.pomZero = self.pomGen(zero)
    self.pomOne = self.pomGen(one)
    self.pomPlus = self.pomGen(plus)
    self.pomMinus = self.pomGen(minus)
    self.pomL = self.pomGen(l)
    self.pomR = self.pomGen(r)
    
  
  def pomGen(self, state):
    return (1/3)* np.outer(state, state.conj())
  
  def pomProbability(self, state,pom):
    prob = np.matrix.trace(pom@state) 
    return prob

  def measure(self, state):
    n = np.random.randint(0,3) #0--> 0>,1> /1--> +>,->/ 2--> L>,R>
    if n == 0:
      p1 = self.pomProbability(state,self.pomZero);
      p2 = self.pomProbability(state,self.pomOne);
      if p1 > p2:
        return 0,n
      else:
        return 1,n
    elif n == 1:
      p1 = self.pomProbability(state,self.pomPlus);
      p2 = self.pomProbability(state,self.pomMinus);
      if p1 > p2:
        return 0,n
      else:
        return 1,n
    elif n == 2:
      p1 = self.pomProbability(state,self.pomL);
      p2 = self.pomProbability(state,self.pomR);
      if p1 > p2:
        return 0,n
      else:
        return 1,n
    
  def measureQbits(self, stateList):
    self.bits = []
    self.usedBasis = []
    for i in range(len(stateList)):
      b,n = self.measure(stateList[i])
      self.bits.append(b)
      self.usedBasis.append(n)
      
  def siftedKey(self, compared):
    self.siftedKey = []
    for i in range(len(compared)):
      if compared[i] == True: 
        self.siftedKey.append(self.bits[i])
        
#   Returns True if the error rate is greater than 5% 
# meaning that an evesdroper was detected.      
  def errorCheck(self, index, bits):
    rate = 0
    ind = 0
    for i in index:
      if self.siftedKey[i] != bits[ind]: 
        rate+= 1 
      ind+=1  
      self.siftedKey[i] = None
    print(rate, len(bits))  
    self.errorRate = rate/len(bits)  
    self.siftedKey = [x for x in self.siftedKey if x is not None]    
    return (rate > (len(index) * 0.05))
  
  def shuffleSiftedKeyKey(self, state):
    np.random.set_state(state)
    np.random.shuffle(self.siftedKey)
    
    

In [0]:
def density(x):
  """ returns the density matrix of all the elements in x  """

  
  p = [] #list to be returned
  for q  in x:
    # |q><q|
    p.append(np.outer(q, q.conj())) #density matrix for each qubit 
    
  return p
  
def transmission(p,q):
    """ returns nosy density matrices given the factor q of noise level """
    t = [] #list to be returned
    for rho in p:
       t.append((1-q)*rho + (q/3)*(X@rho@X + Y@rho@Y + Z@rho@Z))
    
    return t

In [0]:
def 

In [0]:
a = Alice()
b = Bob()

qbits = a.genQbits(20)
# print("Alice bits")
# print(a.bits)
print( )
# print("Alice Qbits")
# print(qbits)
# print( )

# print("Alice Usedbasis")
# print(a.usedBasis)
# print( )
p = density(qbits)
# print("Density")
# # print(p)
# print( )
t = transmission(p,0.6)
# print("Transmission")
# # print(t)
# print( "T SIZE")
# # print(len(t))
# print( )

b.measureQbits(t)

# print("Bob's usedBasis")
# # print(b.usedBasis)
# print("Bob's bits")
# print(b.bits)

compared = a.compareBasis(b.usedBasis)
# for i in compared:
#   print(i)

b.siftedKey(compared)
# print("B raw key " + str(len(b.siftedKey)))
# print("A raw key " + str(len(a.siftedKey)))

index1,bits = a.errorCheck()
res= b.errorCheck(index1,bits)
# print(res)

print("A raw key " + str(a.siftedKey))
print("B raw key " + str(b.siftedKey))
print( )
# state = a.shuffleSiftedKey()
# print("A raw key " + str(a.siftedKey))
# b.shuffleSiftedKey(state)
# print("B raw key " + str(b.siftedKey))
print("Error rate " + str(b.errorRate))





2
0 2
A raw key [0, 1, 0, 0, 1, 0, 0]
B raw key [0, 1, 0, 0, 1, 0, 0]

Error rate 0.0
